In [1]:
# Dependencies
import pandas as pd

In [2]:
# Create DF
df = pd.read_csv("Resources/purchase_data.csv")

In [3]:
# Create Groups
gender_group = df.groupby(['Gender'])
sn_groups = df.groupby(['SN'])
item_groups = df.groupby(['Item ID'])

In [4]:
### Player Count
player_df = pd.DataFrame({"Total Players":df.SN.unique().shape[0]}, index=[0])
player_df

,Total Players
0,576


In [5]:
### Purchasing Analysis (Total)
purchase_analysis = pd.DataFrame({
    "Unique Items": df['Item ID'].unique().shape[0],
    "Average Price": round(df.Price.mean(),2),
    "Number of Purchases": df.shape[0],
    "Total Revenue": df.Price.sum(),
}, index=[0])
purchase_analysis

,Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.05,780,2379.77


In [6]:
### Gender Demographics
gender_counts = (gender_group['SN'].unique()).str.len()
gender_total = gender_counts.sum(axis = 0, skipna = True)

gender_demographics = pd.DataFrame({
    "Total Count":gender_counts[:],
    "Percentage of Players":round(((gender_counts[:]/gender_total)*100),2),
})
gender_demographics

,Total Count,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [7]:
### Purchasing Analysis (Gender)
purchasing_analysis_df = pd.DataFrame({
    'Genders':[i][0],
    'Purchase Count':[df[df['Gender'] == i].shape[0]][0], 
    'Avg. Purchase Price':round([df[df['Gender'] == i]['Price'].mean()][0],2),
    'Tot. Purchase Value':[df[df['Gender'] == i]['Price'].sum()][0],
    'Avg. Total':round([df[df['Gender'] == i]['Price'].sum()][0]/gender_counts[i],2),
} for i in gender_counts.index)
purchasing_analysis_df.set_index(['Genders'])

,Purchase Count,Avg. Purchase Price,Tot. Purchase Value,Avg. Total
Genders,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [8]:
### Age Demographics
age_bins = [0]+[10+5*i for i in range(7)]+[100]
upper = age_bins[1:-1]
lower = age_bins[:-2]
age_labels = ["{0}-{1}".format(l, u-1) for l,u in zip(lower, upper)]+["40+"]
df['ages'] = pd.cut(df['Age'],age_bins,right=False,labels=age_labels)
ages_groups = df.groupby("ages")
ages_counts = ages_groups['SN'].unique().map(len)

ages_df = pd.DataFrame({
    "Age Counts":ages_counts,
    "Pct. of Total":round(ages_counts/ages_counts.sum()*100,2),
})
ages_df

,Age Counts,Pct. of Total
ages,,
0-9,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [9]:
### Purchasing Analysis (Age)
age_demo_counts = ages_groups['SN'].size()
age_demo_average = ages_groups['Price'].mean()
age_demo_total = ages_groups['Price'].sum()

age_demos_df = pd.DataFrame({
    "Age Counts":age_demo_counts,
    "Avg. Purchase Price":round(age_demo_average,2),
    "Tot. Purchase Value":age_demo_total,
    "Avg Total Purchase per Person":round(age_demo_total/ages_counts,2),
})
age_demos_df

,Age Counts,Avg. Purchase Price,Tot. Purchase Value,Avg Total Purchase per Person
ages,,,,
0-9,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


In [10]:
### Top Spenders
sn_df = pd.DataFrame({
    "Purchase Count":sn_groups['SN'].count(),
    "Avg. Purchase Price":round(sn_groups['Price'].mean(),2),
    "Tot. Purchase Value":sn_groups["Price"].sum(),
})

sn_df.sort_values(by=['Tot. Purchase Value'], ascending=False).head()

,Purchase Count,Avg. Purchase Price,Tot. Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [11]:
### Most Popular Items
item_df = pd.DataFrame({
    "Item Name": item_groups['Item Name'].first(),
    "Purchase Count": item_groups['Item ID'].count(),
    "Item Price": item_groups['Price'].first(),
    "Tot. Purchase Value": item_groups['Price'].sum(),
})

item_df.sort_values(by=['Purchase Count'], ascending=False).head()

,Item Name,Purchase Count,Item Price,Tot. Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [12]:
### Most Profitable Items
item_df.sort_values(by=['Tot. Purchase Value'], ascending=False).head()

,Item Name,Purchase Count,Item Price,Tot. Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
